In [2]:
import numpy as np

In [57]:
fpath = "F:\\pp1_data_processing\\2023-06-11 bluetooth data.txt"
with open(fpath, 'r') as f:
    print("adr\tbank\tval\t|\tadrbin\tbankbin\tvalbin\t\trawhex")
    print("---\t----\t---\t|\t------\t----\t------\t\t------")
    i = 0
    for line in f.readlines():
        linehex = int(line, base=16)
        bank = 'A' if (linehex & (1<<12))>0 else 'B'
        # OR with 0x8000 to make the bin string a fixed length
        # bit 15 is unused by firmware so this is fine for conversions
        adrbin = bin((linehex & int('0x0F00', base=16)) | int('0x8000', base=16))
        # reverse adrbin because it is in big endian
        adr = int(adrbin[9:5:-1], base=2)
        
        valbin = bin((linehex & int('0x00FF', base=16))| int('0x8000', base=16))
        # reverse val because it is in big endian
        val = int(valbin[-1:9:-1], base=2)
        print(f"{adr}\t{bank}\t{val}\t|\t{adrbin[9:5:-1]}\t{0 if bank=='A' else 1}\t{valbin[-1:9:-1]}\t{hex(linehex)}")
        i += 1


adr	bank	val	|	adrbin	bankbin	valbin		rawhex
---	----	---	|	------	----	------		------
8	B	255	|	1000	1	11111111	0x21ff
15	A	255	|	1111	0	11111111	0x1fff
7	B	64	|	0111	1	01000000	0x2e02
7	A	130	|	0111	0	10000010	0x1e41
0	B	20	|	0000	1	00010100	0x2028
5	A	255	|	0101	0	11111111	0x1aff
4	A	0	|	0100	0	00000000	0x1200
5	B	0	|	0101	1	00000000	0x2a00
13	B	247	|	1101	1	11110111	0x2bef
13	A	247	|	1101	0	11110111	0x1bef
0	B	0	|	0000	1	00000000	0x2000
0	A	0	|	0000	0	00000000	0x1000
8	A	63	|	1000	0	00111111	0x11fc
12	A	0	|	1100	0	00000000	0x1300
4	B	255	|	0100	1	11111111	0x22ff
4	A	255	|	0100	0	11111111	0x12ff
12	B	9	|	1100	1	00001001	0x2390
12	B	14	|	1100	1	00001110	0x2370
9	B	126	|	1001	1	01111110	0x297e
11	B	126	|	1011	1	01111110	0x2d7e
3	B	126	|	0011	1	01111110	0x2c7e
5	B	126	|	0101	1	01111110	0x2a7e
1	A	0	|	0001	0	00000000	0x1800
8	B	255	|	1000	1	11111111	0x21ff
9	A	255	|	1001	0	11111111	0x19ff
7	B	64	|	0111	1	01000000	0x2e02
7	A	130	|	0111	0	10000010	0x1e41
0	B	20	|	0000	1	00010100	0x2028
5	